 1. IMPORTAMOS LAS LIBRESIAS


In [1]:
import pandas as pd
import numpy as np
import ast
import json

In [2]:
# Se realiza la carga de los datasets movie y credits
data_movie = pd.read_csv("Dataset\movies_dataset.csv")
data_credits = pd.read_csv("Dataset\credits.csv")



C:\Users\angel\AppData\Local\Temp\ipykernel_8604\526186225.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data_movie = pd.read_csv("Dataset\movies_dataset.csv")


2. COMPRENDEMOS LOS DATOS

In [3]:

# Revisamos el numero total de registros de los datasets
print(data_movie.shape)
print(data_credits.shape)


(45466, 24)
(45476, 3)


In [4]:
# checkeamos los tipos de datos de ambos datasets
data_movie.info()
data_credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

3. PREPARACION DE LOS DATOS

In [5]:
# se miran los nulos por columnas
data_movie.isnull().sum()

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

In [6]:
#Para ahora rellenar los valores nulos en revenue y budget
data_movie['revenue'] = data_movie['revenue'].fillna(0)
data_movie['budget'] = data_movie['budget'].fillna(0)

In [7]:
# se eliminará valores nulos de release date
data_movie.dropna(subset=['release_date'], inplace=True)


In [8]:
# Se guardan solo los registron que no tengan release_date nula
data_movie = data_movie[data_movie['release_date'].notna()]

In [9]:
# Revisamos otra vez los nulos despues de los cambios realizados
data_movie.isna().sum()

adult                        0
belongs_to_collection    40888
budget                       0
genres                       0
homepage                 37610
id                           0
imdb_id                     14
original_language           11
original_title               0
overview                   941
popularity                   2
poster_path                339
production_companies         0
production_countries         0
release_date                 0
revenue                      0
runtime                    249
spoken_languages             3
status                      83
tagline                  24981
title                        3
video                        3
vote_average                 3
vote_count                   3
dtype: int64

In [10]:
# Se hace el cambio de formato de fecha a AAAA-mm-dd
data_movie['release_date'] = pd.to_datetime(data_movie['release_date'], format='%Y-%m-%d', errors='coerce')

In [11]:
# Se crea una nueva columna 'release_year', donde se extrae el año de estreno
data_movie['release_year'] = data_movie['release_date'].dt.year
data_movie['release_year']

0        1995.0
1        1995.0
2        1995.0
3        1995.0
4        1995.0
          ...  
45460    1991.0
45462    2011.0
45463    2003.0
45464    1917.0
45465    2017.0
Name: release_year, Length: 45379, dtype: float64

In [12]:
# se borran los valores no validos en budget
data_movie = data_movie[pd.to_numeric(data_movie['budget'], errors='coerce').notnull()]

In [13]:
# se convierte budget y revenue a tipo de dato float
data_movie['budget'] = data_movie['budget'].astype(np.float64)
data_movie['revenue'] = data_movie['revenue'].astype(np.float64)

In [14]:
# Se realiza el calculo del retorno en nueva columna
data_movie['return'] = data_movie.apply(lambda x: x['revenue'] / x['budget'] if x['budget'] != 0 else 0, axis=1)

In [15]:
# cambiamos el tipo de datos de popularity
data_movie['popularity'] = data_movie['popularity'].astype('float')

In [16]:
# Examinamos los repetidos para su posterior eliminacion
print(data_movie.duplicated().sum())
print(data_credits.duplicated().sum())

17
37


In [17]:
# Eliminamos duplicados
data_movie = data_movie.drop_duplicates()
data_credits = data_credits.drop_duplicates()

In [18]:
# Se requiere revisar los duplicados en 'id' 
print(data_movie['id'].duplicated().sum())
print(data_credits['id'].duplicated().sum())



13
7


In [19]:
# Elimino los duplicados en 'id' 
data_movie = data_movie.drop_duplicates(subset = 'id')
data_credits = data_credits.drop_duplicates(subset='id')

In [20]:
# Se uniran los dos datasets para conformar uno solo 
# se nesecita cambiar el tipo de dato para su posterior union
data_movie['id'] = data_movie['id'].astype('int64')

In [21]:
#Se eliminan columnas que no serán de uso
data_movie.drop(columns=['video','imdb_id','adult','original_title','poster_path','homepage'], inplace = True)


In [22]:
# se establece la union de los dataset movie y credits para un solo dataset
d_concat = data_movie.merge(data_credits, on='id', how = 'inner')
d_concat

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,...,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,return,cast,crew
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0,1995.0,12.451801,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,NaN,65000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,1995.0,4.043035,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,A family wedding reignites the ancient feud be...,11.712900,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,1995.0,0.000000,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."
3,NaN,16000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,1995.0,5.090760,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de..."
4,"{'id': 96871, 'name': 'Father of the Bride Col...",0.0,"[{'id': 35, 'name': 'Comedy'}]",11862,en,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,1995.0,0.000000,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45340,NaN,0.0,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",30840,en,"Yet another version of the classic epic, with ...",5.683753,"[{'name': 'Westdeutscher Rundfunk (WDR)', 'id'...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1991-05-13,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Robin Hood,5.7,26.0,1991.0,0.000000,"[{'cast_id': 1, 'character': 'Sir Robert Hode'...","[{'credit_id': '52fe44439251416c9100a899', 'de..."
45341,NaN,0.0,"[{'id': 18, 'name': 'Drama'}]",111109,tl,An artist struggles to finish his work while a...,0.178241,"[{'name': 'Sine Olivia', 'id': 19653}]","[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",2011-11-17,...,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,9.0,3.0,2011.0,0.000000,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de..."
45342,NaN,0.0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",67758,en,"When one of her hits goes wrong, a professiona...",0.903007,"[{'name': 'American World Pictures', 'id': 6165}]","[{'iso_3166_1': 'US', 'name': 'United S

In [23]:
# El dataset nos muestra que tiene campos anidados, se revisara como estan compuestos 
# para su posterior desanidado
print(d_concat['belongs_to_collection'][0])
print(d_concat['genres'][0])
print(d_concat['production_companies'][0])
print(d_concat['production_countries'][0])
print(d_concat['spoken_languages'][0])
print(d_concat['cast'][0])
print(d_concat['crew'][0])

{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}
[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]
[{'name': 'Pixar Animation Studios', 'id': 3}]
[{'iso_3166_1': 'US', 'name': 'United States of America'}]
[{'iso_639_1': 'en', 'name': 'English'}]
[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSd

In [24]:
# Desanidamos la columna 'belongs_to_collection'
d_concat['belongs_to_collection'] = d_concat['belongs_to_collection'].apply(lambda x:ast.literal_eval(x)['name'] if isinstance(x,str) else'').tolist()
d_concat

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,...,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,return,cast,crew
0,Toy Story Collection,30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0,1995.0,12.451801,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,,65000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,1995.0,4.043035,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,Grumpy Old Men Collection,0.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,A family wedding reignites the ancient feud be...,11.712900,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,1995.0,0.000000,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."
3,,16000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,1995.0,5.090760,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de..."
4,Father of the Bride Collection,0.0,"[{'id': 35, 'name': 'Comedy'}]",11862,en,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,1995.0,0.000000,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45340,,0.0,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",30840,en,"Yet another version of the classic epic, with ...",5.683753,"[{'name': 'Westdeutscher Rundfunk (WDR)', 'id'...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1991-05-13,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Robin Hood,5.7,26.0,1991.0,0.000000,"[{'cast_id': 1, 'character': 'Sir Robert Hode'...","[{'credit_id': '52fe44439251416c9100a899', 'de..."
45341,,0.0,"[{'id': 18, 'name': 'Drama'}]",111109,tl,An artist struggles to finish his work while a...,0.178241,"[{'name': 'Sine Olivia', 'id': 19653}]","[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",2011-11-17,...,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,9.0,3.0,2011.0,0.000000,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de..."
45342,,0.0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",67758,en,"When one of her hits goes wrong, a professiona...",0.903007,"[{'name': 'American World Pictures', 'id': 6165}]","[{'iso_3166_1': 'US', 'name': 'United States o...",2003-08-01,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game 

In [25]:
# extraemos los generos especificos de las peliculas en el campo 'genres'
d_concat['genres']= d_concat['genres'].str.replace("'",'"')
d_concat['genres'] = d_concat['genres'].apply(lambda x:[item['name'] for item in json.loads(x)]).tolist()
d_concat.head(3)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,...,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,return,cast,crew
0,Toy Story Collection,30000000.0,"[Animation, Comedy, Family]",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0,1995.0,12.451801,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,,65000000.0,"[Adventure, Fantasy, Family]",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,1995.0,4.043035,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,Grumpy Old Men Collection,0.0,"[Romance, Comedy]",15602,en,A family wedding reignites the ancient feud be...,11.712900,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,1995.0,0.000000,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."


In [26]:
# Creamos una funcionn para desanidar 
# los campos 'Production_companies','Production_countries' y 'spoken_languages'
def desanidar(a):
    if pd.isna(a):
        return none
    else:
        b = ast.literal_eval(a)
        if isinstance(b,list):
            return [i['name'] for i in b]
            return none

In [27]:
#Aplicamos la funcion 'desanidar' en 'Production_companies'
d_concat['production_companies'] = d_concat['production_companies'].apply(desanidar)
d_concat['production_companies'].head(3)

0                            [Pixar Animation Studios]
1    [TriStar Pictures, Teitler Film, Interscope Co...
2                       [Warner Bros., Lancaster Gate]
Name: production_companies, dtype: object

In [28]:
#Aplicamos la funcion 'desanidar' en 'Production_countries'
d_concat['production_countries'] = d_concat['production_countries'].apply(desanidar)
d_concat['production_countries'].head(3)

0    [United States of America]
1    [United States of America]
2    [United States of America]
Name: production_countries, dtype: object

In [29]:
#Aplicamos la funcion 'desanidar' a 'spoken_languages'
d_concat['spoken_languages'] = d_concat['spoken_languages'].apply(desanidar)
d_concat['spoken_languages'].head(3)

0              [English]
1    [English, Français]
2              [English]
Name: spoken_languages, dtype: object

In [30]:
#Aplicamos la funcion 'desanidar' a 'cast'
d_concat['cast'] = d_concat['cast'].apply(desanidar)
d_concat['cast'].head(3)

0    [Tom Hanks, Tim Allen, Don Rickles, Jim Varney...
1    [Robin Williams, Jonathan Hyde, Kirsten Dunst,...
2    [Walter Matthau, Jack Lemmon, Ann-Margret, Sop...
Name: cast, dtype: object

In [31]:
#Para el campo 'crew' requerimos solo el nombre del director, y en este esta referencia en job 
# se nesecita crear una nueva funcion para extraer el nombre
def desanidar_crew(a):
    b = ast.literal_eval(a)
    for i in b:
        if i.get('job') == 'Director':
            return i.get('name')
    return None


In [32]:
# Aplicamos la funcion 'desanidar_crew' a 'crew' y ademas se crea un nuevo campo 'director'
d_concat['director'] = d_concat['crew'].apply(desanidar_crew) 
d_concat.head(3)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,...,status,tagline,title,vote_average,vote_count,release_year,return,cast,crew,director
0,Toy Story Collection,30000000.0,"[Animation, Comedy, Family]",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,[Pixar Animation Studios],[United States of America],1995-10-30,...,Released,NaN,Toy Story,7.7,5415.0,1995.0,12.451801,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",John Lasseter
1,,65000000.0,"[Adventure, Fantasy, Family]",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],1995-12-15,...,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,1995.0,4.043035,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",Joe Johnston
2,Grumpy Old Men Collection,0.0,"[Romance, Comedy]",15602,en,A family wedding reignites the ancient feud be...,11.712900,"[Warner Bros., Lancaster Gate]",[United States of America],1995-12-22,...,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,1995.0,0.000000,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",Howard Deutch


In [33]:
#Creo un nuevo data set solo con los campos requeridos para realizar unas funciones
# especificas para este proyecto
Dataset_Final = d_concat[['id','title','genres','director','belongs_to_collection','production_companies','production_countries',
                 'original_language','release_date','release_year','runtime','vote_average',
                 'vote_count','popularity','revenue','budget','return']] 

In [34]:
Dataset_Final.head(3)

,id,title,genres,director,belongs_to_collection,production_companies,production_countries,original_language,release_date,release_year,runtime,vote_average,vote_count,popularity,revenue,budget,return
0,862,Toy Story,"[Animation, Comedy, Family]",John Lasseter,Toy Story Collection,[Pixar Animation Studios],[United States of America],en,1995-10-30,1995.0,81.0,7.7,5415.0,21.946943,373554033.0,30000000.0,12.451801
1,8844,Jumanji,"[Adventure, Fantasy, Family]",Joe Johnston,,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],en,1995-12-15,1995.0,104.0,6.9,2413.0,17.015539,262797249.0,65000000.0,4.043035
2,15602,Grumpier Old Men,"[Romance, Comedy]",Howard Deutch,Grumpy Old Men Collection,"[Warner Bros., Lancaster Gate]",[United States of America],en,1995-12-22,1995.0,101.0,6.5,92.0,11.712900,0.0,0.0,0.000000


In [36]:
Dataset_Final.to_csv(r"C:\Users\angel\Desktop\RATG\Datasciencie\Henry-Proyect\Dataset_Final.csv")

In [35]:
Dataset_Final.columns

Index(['id', 'title', 'genres', 'director', 'belongs_to_collection',
       'production_companies', 'production_countries', 'original_language',
       'release_date', 'release_year', 'runtime', 'vote_average', 'vote_count',
       'popularity', 'revenue', 'budget', 'return'],
      dtype='object')

In [40]:
d_concat.to_csv(r"C:\Users\angel\Desktop\RATG\Datasciencie\Nuevo Proyecto\Proyecto-Individual\dataset\Dataset_PI.csv")